In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.impute import SimpleImputer

In [52]:
mi_df =  pd.read_csv(
    'data/01_data_cruda/MI_data.csv', 
    header=0, 
    sep=","
)

There  are  four  possible  time  moments  for  complication  prediction:  on  base  of  the information known at

1.the time of admission to hospital:all input columns (2-112) except 93, 94, 95, 100, 101, 102, 103, 104, 105can be usedfor prediction;

2.the end of the first day(24 hours after admission to the hospital):all input columns (2-112) except 94, 95, 101, 102, 104, 105 can be usedfor prediction;

3.the end of the second day (48 hours after admission to the hospital) all input columns (2-112) except 95, 102, 105 can be usedfor prediction;

4.the end of the third day (72 hours after admission to the hospital) all input columns (2-112) can be usedfor prediction.

---




In [55]:
# Hacemos referencia a las columnas 2-112 (Variables que pueden ser usadas para la predicción)

# Referenciamos columnas con indice de 0 a 111
mi_comp_df = mi_df.iloc[:,112:124]
mi_df = mi_df.iloc[:,0:112]

mi_df = pd.concat([mi_df, mi_comp_df], axis=1, join='inner')
# Removemos columnas 93, 94 , 95, 101, 102, 103, 104, 105
mi_df.drop(mi_df.columns[[1-1, 93-1, 94-1 , 95-1, 101-1, 102-1, 103-1, 104-1, 105-1]],axis = 1, inplace=True)

### 1. Removemos columnas

#### 1.1 Remover columnas que tienen un porcentaje de valores únicos superior al 70%

In [67]:
#Determinamos aquellas columnas (variables) que tengan un porcentaje de filas (registros) únicas por cada columna (variable) mayor al valor de 0.7
threshold = 0.7

unique_percentages = mi_df.nunique() / len(mi_df)

criteria = unique_percentages > threshold

columns_to_filter = unique_percentages[criteria].keys()

#Eliminamos las columnas (variables) seleccionadas en el paso anterior. Al aplicar "drop" obtenemos un nuevo "DataFrame", pero con el parámetro
#inplace igual a "True" podemos realizar dicha operación de eliminación y sobre-escribir el "DataFrame" del cual estamos eliminando con el
#nuevo "DataFrame" obtenido. El parámetro axis permite seleccionar el eje respecto al cual aplicar la operación "drop". Los valores válidos
#son [0 o "index"] y [1 o "columns"]
mi_df.drop(columns_to_filter, axis=1, inplace=True)

### 2. Tratamiento de datos perdidos

In [68]:
from collections import defaultdict
from sklearn import preprocessing 

In [69]:
### El dataset solo contiene variables numéricas y no contiene variables categóricas que tengan que ser codificadas

# Agrupando columnas por tipo de datos
tipos = mi_df.columns.to_series().groupby(mi_df.dtypes).groups

# Armando lista de columnas categóricas
cint = tipos[np.dtype('int64')]
# Armando lista de columnas numéricas
cfloat = tipos[np.dtype("float64")]

In [70]:
## Para columnas que tengan > 20% de valores perdidos: Eliminar
cols_mayor_a_20 = [col for col in mi_df.columns if ((mi_df[col].isnull().sum() / len(mi_df)*100) > 20.0)]

mi_df.drop(
    cols_mayor_a_20, 
    axis = 1, 
    inplace=True
)

In [71]:
## Para columnas que tengan entre 15% y 20% de valores perdidos: Imputar con modelos
cols_entre_15_y_20 = [
    col for col in mi_df.columns if 
        (((mi_df[col].isnull().sum() / len(mi_df)*100))>=15 and  
        ((mi_df[col].isnull().sum() / len(mi_df)*100))<=20)
]

In [72]:
#sns.pairplot(mi_df)

In [73]:
cols_entre_15_y_20

['S_AD_ORIT', 'D_AD_ORIT', 'ALT_BLOOD', 'AST_BLOOD']

In [74]:
## Para columnas que tengan menos de 15% de valores perdidos: Imputar con Media, Moda y Mediana
cols_menor_a_15 = [col for col in mi_df.columns if ((mi_df[col].isnull().sum() / len(mi_df)*100) < 15.0)]

Notas de la libreria SimpleImputer para el parametro "strategy"

- Si es "mean", reemplace los valores faltantes usando la media a lo largo de cada columna. Solo se puede usar con datos numéricos flotantes (float).

- Si es "median", reemplace los valores faltantes usando la mediana a lo largo de cada columna. Solo se puede usar con datos numéricos enteros (int).

- Si es "most_frequent", reemplace falta utilizando el valor más frecuente a lo largo de cada columna. Se puede usar con cadenas o datos numéricos que representan variables categóricas (str/int).

In [75]:
imp_media = SimpleImputer(missing_values=np.nan, #etiqueta de valores nulos
                    strategy='mean') #método o estrategia de imputación
#imp_media es la función con los parámetros ya especificados de nuestra función

imp_moda = SimpleImputer(missing_values=np.nan, #etiqueta de valores nulos
                    strategy='most_frequent') #método o estrategia de imputación
#imp_moda es la función con los parámetros ya especificados de nuestra función

In [76]:
missing_cols_int_lst = [col for col in cols_menor_a_15 if col in cint]

mi_int_limpio_df = imp_moda.fit_transform(mi_df[missing_cols_int_lst])
mi_int_limpio_df = pd.DataFrame(mi_int_limpio_df, columns=missing_cols_int_lst)

In [77]:
missing_cols_float_lst = [col for col in cols_menor_a_15 if col in cfloat]

mi_float_limpio_df = imp_media.fit_transform(mi_df[missing_cols_float_lst])
mi_float_limpio_df = pd.DataFrame(mi_float_limpio_df, columns=missing_cols_float_lst)

In [83]:
mi_df.drop(
    missing_cols_int_lst+missing_cols_float_lst,
    axis = 1, 
    inplace=True
)
mi_df = pd.concat([mi_df, mi_int_limpio_df], axis=1, join='inner')
mi_df = pd.concat([mi_df, mi_float_limpio_df], axis=1, join='inner')

In [78]:
mi_df["ZSN"].unique()

array([0, 1], dtype=int64)

In [84]:
((mi_df.isnull().sum() / len(mi_df)*100)>0.001).sum()

4